In [1]:
import pathlib
import delta
import json
import os

Let's check if we will work on CPU or GPU:

In [2]:
import tensorflow as tf
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

Set the path to the one you used in `download_test_data_delta`

In [3]:
root = pathlib.Path(pathlib.Path.home(), 
                    'I2ICourse/RawData/project1')

output_dir_gen = pathlib.Path(pathlib.Path.home(), 
                    'I2ICourse/ProcessedData/project1/cib_new/')

(output_dir_gen).mkdir(exist_ok=True)


In [17]:
folder_names = sorted((root / 'cib_new').glob('Pos*'))

for folder in folder_names: print(folder.name)

    

Pos01
Pos02
Pos03
Pos04
Pos05
Pos06
Pos07
Pos08
Pos09


We need to modify the config file for the proper paths

In [19]:
for folder in folder_names:

    posname = folder.name
    
    print('ruuning position ', posname)
    output_dir = output_dir_gen / posname
    (output_dir).mkdir(exist_ok=True)

    try:
        # Load config ('2D' or 'mothermachine'):
        config_file = str((root / 'config_2D.json').resolve())

        with open(config_file) as f:
            data = json.load(f)
            
        old_model_path = 'D:/DeLTA_data/agar_pads'  
        new_model_path = str((root / 'models').resolve())
        data['model_file_seg'] = data['model_file_seg'].replace(old_model_path, new_model_path)    
        data['model_file_track'] = data['model_file_track'].replace(old_model_path, new_model_path)

        data['training_set_seg'] = str((root / 'sets' / '2D' / 'training' / 'segmentation_set').resolve())
        data['training_set_track'] = str((root / 'sets' / '2D' / 'training' / 'tracking_set').resolve())

        data['eval_movie'] = str((root / 'cib_new' / posname).resolve())

        data['save_format'] = ["pickle", "movie"]   

        new_config_file = str((root / 'config_2D_local.json').resolve())
        with open(new_config_file, 'w') as f:
            json.dump(data, f, indent=2)
            
            
        # Load config ('2D' or 'mothermachine'):
        delta.config.load_config(new_config_file)

        # Use eval movie as example (or replace by None for interactive selection):
        file_path = delta.config.eval_movie

        # Init reader (use bioformats=True if working with nd2, czi, ome-tiff etc):
        xpreader = delta.utils.xpreader(
                    file_path,
                    prototype = 'Pos%03i_Frm%03i_Ch%02i.tif',
                    fileorder = 'ptc',
                    filenamesindexing=1
                    )

        # Print experiment parameters to make sure it initialized properly:
        print("""Initialized experiment reader:
            - %d positions
            - %d imaging channels
            - %d timepoints"""%(xpreader.positions, xpreader.channels, xpreader.timepoints)
        )

        # Init pipeline:
        xp = delta.pipeline.Pipeline(xpreader, resfolder=output_dir)   


        # Run it (you can specify which positions, which frames to run etc):
        xp.process()
        
    except:
        print('skipping postion', posname)

ruuning position  Pos01
Loading configuration from: /scicore/home/vanvli0000/vanvli0000/I2ICourse/RawData/project1/config_2D_local.json
Initialized experiment reader:
            - 1 positions
            - 2 imaging channels
            - 91 timepoints
Thu Mar 10 20:10:38 2022, Position 0 - Starting pre-processing
Thu Mar 10 20:10:38 2022, Position 0 - Starting segmentation (91 frames)
Thu Mar 10 20:10:53 2022, Position 0 - Starting tracking (91 frames)
Thu Mar 10 20:10:53 2022, Position 0 - Tracking - frame 0/91 
Thu Mar 10 20:10:53 2022, Position 0 - Tracking - frame 1/91 
Thu Mar 10 20:10:53 2022, Position 0 - Tracking - frame 2/91 
Thu Mar 10 20:10:53 2022, Position 0 - Tracking - frame 3/91 
Thu Mar 10 20:10:53 2022, Position 0 - Tracking - frame 4/91 
Thu Mar 10 20:10:53 2022, Position 0 - Tracking - frame 5/91 
Thu Mar 10 20:10:53 2022, Position 0 - Tracking - frame 6/91 
Thu Mar 10 20:10:53 2022, Position 0 - Tracking - frame 7/91 
Thu Mar 10 20:10:53 2022, Position 0 - Tracki